In [1]:
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(GenomicRanges)
library(naturalsort)
library(data.table)


Loading required package: GenomicFeatures
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: IRange

In [2]:
genes <- genes(TxDb.Hsapiens.UCSC.hg38.knownGene)
head(genes)

GRanges object with 6 ranges and 1 metadata column:
            seqnames               ranges strand |     gene_id
               <Rle>            <IRanges>  <Rle> | <character>
          1    chr19 [58346806, 58362848]      - |           1
         10     chr8 [18391245, 18401213]      + |          10
        100    chr20 [44619522, 44651735]      - |         100
       1000    chr18 [27950966, 28177481]      - |        1000
  100008589    chr21 [ 8213888,  8401980]      + |   100008589
  100009613    chr11 [70072434, 70075348]      - |   100009613
  -------
  seqinfo: 455 sequences (1 circular) from hg38 genome

In [3]:
tsgs <- read.table("../data/Human_TSGs.txt")
tsgs <- tsgs[2:nrow(tsgs),]
names(tsgs) <- c("GeneID","GeneSymbol")
head(tsgs)

,GeneID,GeneSymbol
2,43,ACHE
3,95,ACY1
4,104,ADARB1
5,141,ADPRH
6,142,PARP1
7,185,AGTR1


In [4]:
tsg_ranges<-c()
unfound <- 0
k<-1
for (i in 1:nrow(tsgs)){ # Loop over TSGs
    tmp <- genes[genes$gene_id==tsgs$GeneID[i]] # Get the row of genes matching current TSG
    if (length(tmp) > 0) {
        tsg_ranges$chrom[k] <- runValue(seqnames(tmp))
        tsg_ranges$start[k] <- start(tmp)
        tsg_ranges$end[k]   <- end(tmp)
        k<-k+1
    } else {unfound <- unfound+1}
}
tsg_ranges <- data.frame(tsg_ranges)
tsg_ranges <- tsg_ranges[order(tsg_ranges$chrom),]
tsg_ranges <- tsg_ranges[tsg_ranges$chrom<25,]
tsg_ranges$chrom[tsg_ranges$chrom==23]<-'X'
tsg_ranges$chrom[tsg_ranges$chrom==24]<-'Y'
tsg_ranges$chrom <- paste0('chr',tsg_ranges$chrom)
head(tsg_ranges)

,chrom,start,end
5,chr1,226360691,226408100
11,chr1,21509365,21578412
19,chr1,228082660,228099212
29,chr1,212565334,212620777
60,chr1,24899511,24965121
77,chr1,50967936,50974637


In [6]:
cat(paste(nrow(tsgs)-unfound,'/',nrow(tsgs),'tsgs were found'))
(nrow(tsgs)-unfound)/nrow(tsgs)*100

1166 / 1217 tsgs were found

[1] 95.80937

In [8]:
tmp<-c()
# exann <- read.table('../../Data/humangenome//Homo_sapiens.GRCh38.89.gff3.exonfilt')
cdsann <- exons(TxDb.Hsapiens.UCSC.hg38.knownGene)
# cdsann <- GRanges(seqnames=cdsann$V1,IRanges(cdsann$V2,cdsann$V3))
cdsann <- reduce(cdsann)
tmp$chrom <- as.vector(seqnames(cdsann))
tmp$start <- start(cdsann)
tmp$end <- end(cdsann)
cdsann <- data.frame(tmp)
names(cdsann) <- c('chrom','start','end')
cdsann$istsg <- 0
head(cdsann)

chrom,start,end,istsg
chr1,11874,12227,0
chr1,12595,12721,0
chr1,13221,14409,0
chr1,30366,30503,0
chr1,69091,70008,0
chr1,182393,182746,0


In [9]:
test <- exonsBy(TxDb.Hsapiens.UCSC.hg38.knownGene)

In [10]:
head(test)

GRangesList object of length 6:
$1 
GRanges object with 3 ranges and 3 metadata columns:
      seqnames         ranges strand |   exon_id   exon_name exon_rank
         <Rle>      <IRanges>  <Rle> | <integer> <character> <integer>
  [1]     chr1 [11874, 12227]      + |         1        <NA>         1
  [2]     chr1 [12613, 12721]      + |         3        <NA>         2
  [3]     chr1 [13221, 14409]      + |         5        <NA>         3

$2 
GRanges object with 3 ranges and 3 metadata columns:
      seqnames         ranges strand | exon_id exon_name exon_rank
  [1]     chr1 [11874, 12227]      + |       1      <NA>         1
  [2]     chr1 [12646, 12697]      + |       4      <NA>         2
  [3]     chr1 [13221, 14409]      + |       5      <NA>         3

$3 
GRanges object with 3 ranges and 3 metadata columns:
      seqnames         ranges strand | exon_id exon_name exon_rank
  [1]     chr1 [11874, 12227]      + |       1      <NA>         1
  [2]     chr1 [12595, 12721]      + |

In [27]:
for (i in 1:nrow(cdsann)) {
    
    tmp <- tsg_ranges[tsg_ranges$chrom==cdsann$chrom[i],]
    if (length(which(between(cdsann$start[i],tmp$start,tmp$end)))>0) {
        cdsann$istsg[i] <- 1
    }
    
}

In [28]:
head(cdsann[cdsann$istsg==1,])

,chrom,start,end,istsg
61,chr1,1013467,1013576,1
62,chr1,1013984,1014539,1
112,chr1,1167104,1167198,1
113,chr1,1167863,1167952,1
385,chr1,3652565,3652641,1
386,chr1,3682333,3682430,1


In [29]:
sum(cdsann$end-cdsann$start)/3e9

[1] 0.031718

In [30]:
exann <- cdsann
save(exann, file='../data/exann.rda')